In [ ]:
# Copyright 2022 Google
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Time Crystal Circuit Generation
This notebook covers how Many Body Local Discrete Time Crystal circuit lists are created, from the paper: Observation of Time-Crystalline Eigenstate Order on a Quantum Processor ([arxiv:2107.13571](https://arxiv.org/abs/2107.13571)). 

Quantum computers, and gate-based quantum circuits turn out to be well suited for crafting systems that exhibit time-crystalline behavior. Behavior is crystalline with respect to time if it has some consistent and stable pattern over time. This system's pattern must be resilient against perturbation, including the introduction of random noise and heat, in the same way that a space-crystalline object, like a diamond, stays in shape if moved or heated. 

The implementation of a time-crystalline system on a quantum computer starts with a set of qubits that are connected in a single chain. This serves as a many body local system which, at the low superconducting temperature that the qubits are run at, becomes resistant to changes cause by small amounts of introduced heat. The heat's energy is spread across the qubits, thin enough that it is not sufficient to change the qubits' states. The amount of heat dissipation in the DTC circuits is modeled with the thermalization constant `g`. 

The actual time-crystalline behavior that the DTC circuits implement is perhaps the simplest possible kind of time-structured behavior, oscillation. Each circuit is built with some number of identical $U$-cycles. Time is represented by a circuit list where each circuit is ordered with increasingly many $U$-cycles; each cycle is a discrete time step. The eventual effect of these $U$-cycles, as demonstrated in the paper and in the [Time Crystal Data Analysis](time_crystal_data_analysis.ipynb) notebook, is consistent oscillations of each qubits' polarizations. Each $U$-cycle includes a component to model to model the influence of thermalization, with the constant `g`,  and random noise, with the `local_fields` variables. The behavior of the DTC circuits is shown to be time-crystalline in spite of the consistently introduced randomness. 

## Setup

In [ ]:
!pip install cirq --pre --quiet
try:
    import recirq
except ImportError:
    !pip install --quiet git+https://github.com/quantumlib/ReCirq

In [ ]:
import cirq
import recirq.time_crystals as time_crystals

## Circuit Construction
Each DTC circuit is created with symbolic parameters. Parameter values are supplied near run/simulation time with a `cirq.ParamResolver`, which means the circuit list needs to be generated only once for potentially many different experimental parameter configurations. 

The code below uses an IPython-specific utility to inspect the code of the key function that creates the symbolic circuit list, `recirq.time_crystals.symbolic_dtc_circuit_list()`. 

In [ ]:
import inspect
from IPython.display import Code

Code(inspect.getsource(time_crystals.symbolic_dtc_circuit_list), language="python")

The construction of each circuit is surprisingly short. 

The circuit expects the quantum computer to be in the all-zeros state, and starts with a sequence of `cirq.Y` gates conditioned on the provided `initial state` parameter, after initializing the necessary symbolic variables. 

Each $U$-cycle consists of three moments. First, a moment of `cirq.PhasedXZGate`s, with one for each qubit. Each `cirq.PhasedXZGate` takes the thermalization constant `g` as its X-exponent, and the random variance provided by `local_fields` for its Y-exponent. This moment collectively models the disorder caused by heat and random noise. 

The second and third moments both cause the oscillation behavior and compensate for the first disorder moment. The qubits are connected in a chain, and each qubit pair connection in that chain is coupled with a `cirq.PhasedFSimGate` that uses the parameters `[theta, zetas, chi, gamma, phi]`. To keep gates from overlapping on the same qubit, this chain of gates is split into the second and third moments, such that no two gates share a qubit within each moment. 

Finally, `symbolic_dtc_circuit_list()` builds and returns a list of circuits with $0,1,2,..., cycles$ many $U$-cycles in them. 

In [ ]:
qubits = [cirq.GridQubit(0, i) for i in range(4)]
circuit_list = time_crystals.symbolic_dtc_circuit_list(qubits, 2)
for circuit in circuit_list:
    print("\ncircuit of length " + str(len(circuit)) + "\n")
    print(circuit)

After the initial line of `cirq.Y` gates, each consecutive circuit in the list has an additional cycle of `cirq.PhasedXZGate`s, followed by the chain of `cirq.PhasedFSimGate`s on alternating qubit pairs. Each cycle of three moments becomes one time step in the later analysis of stable oscillations over time. 

The next step is to perform experiments to collect evidence of the time-crystalline behavior of the quantum state's polarizations. See the [Time Crystal Data Collection](time_crystal_data_collection.ipynb) notebook for the experiments, and the [Time Crystal Data Analysis](time_crystal_data_analysis.ipynb) notebook for the graphed data and results. 